In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cycler
from matplotlib.ticker import StrMethodFormatter
from matplotlib.axis import Axis
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
import sqlite3 as sq
from functools import reduce
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon

In [2]:
#string for naming files
county = 'WilsonCo'

In [3]:
#functions
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## Census Bureau

In [4]:
conn = sq.connect('../../../Data-Pipelines/Outputs/CensusBureau.db')

#2000

#SF12000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2000]', conn)
sf12000 = pd.DataFrame(sql_query)
sf12000.drop(columns = 'Source', inplace = True)
#SF32000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF3_2000]', conn)
sf32000 = pd.DataFrame(sql_query)
sf32000.drop(columns = 'Source', inplace = True)
#TIGER2000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2000]', conn)
TIGER2000 = pd.DataFrame(sql_query)
TIGER2000.drop(columns = 'Source', inplace = True)
dfs2000 = [TIGER2000, sf12000, sf32000]
twothousand= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2000)
twothousand = twothousand.set_index(['NAME', 'GEO_ID'])
twothousandsuff = twothousand.add_suffix(' 2000_1YR')


#2010


#ACS52010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2010]', conn)
ACS52010 = pd.DataFrame(sql_query)
ACS52010.drop(columns = 'Source', inplace = True)
twothousandten = ACS52010.set_index(['NAME', 'GEO_ID'])
twothousandten = twothousandten.drop(columns = 'index')
twothousandtenfivesuff = twothousandten.add_suffix(' 2010_5YR')

#SF12010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2010]', conn)
sf12010 = pd.DataFrame(sql_query)
sf12010.drop(columns = 'Source', inplace = True)
#TIGER2010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2010]', conn)
TIGER2010 = pd.DataFrame(sql_query)
TIGER2010.drop(columns = 'Source', inplace = True)
dfs2010 = [TIGER2010, sf12010]
twothousandten= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2010)
twothousandten = twothousandten.fillna(0)
twothousandten = twothousandten.set_index(['NAME', 'GEO_ID'])
twothousandtensuff = twothousandten.add_suffix(' 2010_1YR')

#"CURRENT"
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#TIGER2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2020]', conn)
TIGER2020 = pd.DataFrame(sql_query)
TIGER2020.drop(columns = 'Source', inplace = True)
dfs2020 = [TIGER2020, PL2020]
twothousandtwenty = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2020)
twothousandtwenty = twothousandtwenty.fillna(0)
twothousandtwenty = twothousandtwenty.set_index(['NAME', 'GEO_ID'])
twothousandtwentysuff = twothousandtwenty.add_suffix(' 2020_1YR')
#ACS52022
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_1]', conn)
ACS52022_1 = pd.DataFrame(sql_query)
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2022_2]', conn)
ACS52022_2 = pd.DataFrame(sql_query)
ACS52022 = ACS52022_1.merge(ACS52022_2, on = ['NAME', 'GEO_ID', 'Source'])
ACS52022.drop(columns = 'Source', inplace = True)
ACS52022 = ACS52022.fillna(0)
ACS52022 = ACS52022.set_index(['NAME', 'GEO_ID'])
twothousandtwentytwosuff = ACS52022.add_suffix(' 2022_5YR')

#PEP Population and Housing Units
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PEP_PopHU_20202023]', conn)
pep = pd.DataFrame(sql_query)
pep = pep.loc[pep['Year'] == '2023']
pep = pep.drop(columns = ['Source', 'Time Frame', 'Housing Units % Change', 'Housing Units Change', 'Population % Change', 'Population Change', 'Year'])
pep = pep.fillna(0)
pep = pep.set_index(['NAME', 'GEO_ID'])
pepsuff = pep.add_suffix(' 2023_PEP')

In [5]:
geos = ['Wilson County']

In [6]:
dfs = [twothousandsuff, twothousandtensuff, twothousandtenfivesuff, twothousandtwentysuff, twothousandtwentytwosuff, pepsuff]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'], how='outer'), dfs)

In [7]:
dfsuff = dfsuff.reset_index().set_index('NAME').transpose()
dfsuff = dfsuff.rename(columns = namestocommon)
dfsuff = dfsuff[geos]
dfsuff = dfsuff.transpose().reset_index()

In [8]:
cols = list(dfsuff.columns)
cols.remove('NAME')
cols.remove('GEO_ID')
dfsuff[cols] = dfsuff[cols].astype(float)

In [9]:
cols = ['NAME', 'Population 2010_1YR', 'Population 2020_1YR', 'Population 2023_PEP', 'Minority 2020_1YR', 'Minority % 2020_1YR', 'Age:School Age 5 to 18 2022_5YR', 
        'Age%:School Age 5 to 18 2022_5YR', 
        'Poverty: Population Below Poverty Level 2022_5YR', 'Poverty%: Below Poverty Level 2022_5YR', 'Age:55 and Older 2022_5YR', 'Age%:55 and Older 2022_5YR', 
        'Age:65 and Older 2022_5YR', 'Age%:65 and Older 2022_5YR', 'Units:Total Series 2010_5YR', 'Units:Total Series 2022_5YR', 'Housing Units 2023_PEP',  
        'Units:1 Unit, Detached 2022_5YR', 'Units:1 Unit, Attached 2022_5YR', 'Units:Mobile Home 2022_5YR',
        'Units%:1 Unit, Detached 2022_5YR', 'Units%:1 Unit, Attached 2022_5YR', 'Units%:Mobile Home 2022_5YR', 'Median Household Income 2022_5YR', 
        'CB:Cost Burdened Homeowners 2022_5YR', 'CB:Cost Burdened Renters 2022_5YR',
        'CB%: Cost Burdened Homeowners 2022_5YR', 'CB%:Cost Burdened Renters 2022_5YR', 'Average Household Size 2022_5YR', 
        'Tenure:Owners 2022_5YR', 'Tenure%:Owners 2022_5YR',
        'Tenure:Renters 2022_5YR', 'Tenure%:Renters 2022_5YR', 'Square Acres 2020_1YR']

In [10]:
df = dfsuff[cols]

In [11]:
df['Population Increase 2010 to 2020 1 YR EST'] = df['Population 2020_1YR'] - df['Population 2010_1YR']
months = 12 * 10
df['Population Gained Per Month 2010 to 2020 1 YR EST'] = df['Population Increase 2010 to 2020 1 YR EST']/months
df['Housing Unit Increase 2010 to 2020 1 YR EST'] = df['Units:Total Series 2022_5YR'] - df['Units:Total Series 2010_5YR']
df['Housing Units Gained Per Month 2010 to 2020 1 YR EST'] = df['Housing Unit Increase 2010 to 2020 1 YR EST']/months
df['Current Population 2023 PEP'] = df['Population 2023_PEP']

In [12]:
thelist = [df['Units:1 Unit, Detached 2022_5YR'], df['Units:1 Unit, Attached 2022_5YR'], df['Units:Mobile Home 2022_5YR']]
df['Units:All Multi-Family 2022_5YR'] = df['Units:Total Series 2022_5YR'] - sum(thelist)
df['Units%:All Multi-Family 2022_5YR'] = percent(df['Units:All Multi-Family 2022_5YR'], df['Units:Total Series 2022_5YR'])

In [13]:
df = df.set_index('NAME').transpose()
df

NAME,Wilson County
Population 2010_1YR,113993.000000
Population 2020_1YR,147737.000000
Population 2023_PEP,163674.000000
Minority 2020_1YR,28848.000000
Minority % 2020_1YR,19.526591
Age:School Age 5 to 18 2022_5YR,25973.000000
Age%:School Age 5 to 18 2022_5YR,17.420320
Poverty: Population Below Poverty Level 2022_5YR,12172.000000
Poverty%: Below Poverty Level 2022_5YR,8.290593
Age:55 and Older 2022_5YR,43370.000000


In [14]:
df.to_csv('CensusData_AtaGlance_{}Co.csv'.format(county))

## Jobs

We're going to use BEA here because we had described the industry grouping as "government" here always, so it makes sense since we aren't trying to highlight education or health services or anything...

In [15]:
data = pd.read_csv('../../AdjustedYoYEmpl_BLandNAICS.csv')
data = data.loc[(data['NAME'] == 'Wilson County, Tennessee')]
data = data.loc[(data['Year'] == '2010')|(data['Year'] == '2020')|(data['Year'] == '2023')].reset_index(drop = True)
data = data.drop(columns = ['NAICS Code', 'BEA Line Code', 'Category'])

In [16]:
data['Year'].unique()

array(['2023', '2020', '2010'], dtype=object)

In [17]:
decade = data.loc[((data['Year'] == '2010')|(data['Year'] == '2020'))&(data['Industry'] == 'Total employment (number of jobs)')].reset_index(drop = True)
decade = decade.drop(columns = ['NAICS Employment', 'NAME', 'Industry'])

In [18]:
decade.head()

,Year,BEA Employment
0,2020,79263.0
1,2010,53292.0


In [19]:
decade = decade.set_index('Year').transpose()
decade['Jobs Change 2010 to 2020'] = decade['2020'] - decade['2010']
decade['Jobs % Change 2010 to 2020'] = percentchange(decade['2020'], decade['2010'])
decade['Jobs Gained Per Month 2010 to 2020'] = decade['Jobs Change 2010 to 2020']/months
decade.reset_index(drop = False, inplace = True)
decade = decade.drop(columns = ['index', '2010'])
decade = decade.add_suffix(' BEA')

In [20]:
decade.head()

Year,2020 BEA,Jobs Change 2010 to 2020 BEA,Jobs % Change 2010 to 2020 BEA,Jobs Gained Per Month 2010 to 2020 BEA
0,79263.0,25971.0,48.733393,216.425


In [21]:
decade.to_csv('TotalEmployment_PastDecade_AtaGlance_{}.csv'.format(county))

Current year for breakout

In [22]:
data = data.loc[data['Year'] == '2023'].reset_index(drop = True)
data = data.drop(columns = ['Year', 'NAICS Employment', 'NAME'])
data = data.loc[data['Industry'] != 'Public Administration']
data = data.set_index('Industry').transpose()

In [23]:
data.head()

Industry,Information,Health care and social assistance,Government,"Forestry, fishing, and related activities",Finance and insurance,Farm employment,Educational services,Construction,"Arts, entertainment, and recreation","Agriculture, forestry, fishing and hunting",Administrative and support and waste management and remediation services,Accommodation and food services,Management of companies and enterprises,Manufacturing,"Mining, quarrying, and oil and gas extraction",Other services (except government and government enterprises),Wholesale trade,Utilities,Transportation and warehousing,Total employment (number of jobs),Retail trade,Real estate and rental and leasing,"Professional, scientific, and technical services"
BEA Employment,925.392857,5478.035714,6799.285714,315.070066,4310.142857,1463.642857,1686.857143,6123.25,2077.178571,1778.712923,5036.464286,6851.035714,945.857143,5017.642857,231.408608,5682.571429,3123.516766,243.540275,17127.285714,93902.178571,8906.714286,5780.892857,5776.392857


In [24]:
thelist = [data['Information'], data['Finance and insurance'],
           data['Real estate and rental and leasing'], data['Professional, scientific, and technical services'], 
           data['Management of companies and enterprises'], data['Educational services'], data['Health care and social assistance'], 
           data['Government']]
data['White Collar'] = sum(thelist)

thelist = [data['Mining, quarrying, and oil and gas extraction'], data['Utilities'], data['Construction'], data['Manufacturing'], 
           data['Wholesale trade'], data['Transportation and warehousing']]
data['Blue Collar'] = sum(thelist)

thelist = [data['Retail trade'], data['Accommodation and food services']]
data['Retail & Hospitality'] = sum(thelist)
data = data.drop(columns = ['Retail trade', 'Accommodation and food services'])

thelist = [data['Information'], data['Finance and insurance'],
           data['Real estate and rental and leasing'], data['Professional, scientific, and technical services'], 
           data['Management of companies and enterprises'], data['Administrative and support and waste management and remediation services'], 
           data['Educational services'], data['Health care and social assistance'], 
           data['Arts, entertainment, and recreation'], data['Other services (except government and government enterprises)']]
data['Professional Services'] = sum(thelist)
data = data.drop(columns = ['Information', 'Finance and insurance', 'Real estate and rental and leasing', 'Professional, scientific, and technical services', 
                            'Management of companies and enterprises', 'Administrative and support and waste management and remediation services', 
                            'Educational services', 'Health care and social assistance', 'Arts, entertainment, and recreation', 
                            'Other services (except government and government enterprises)'])

thelist = [data['Forestry, fishing, and related activities'], data['Mining, quarrying, and oil and gas extraction'],
           data['Utilities'], data['Construction'], data['Manufacturing'], data['Wholesale trade'], data['Transportation and warehousing']]
data['Industrial, Manufacturing, Utilities, & Logistics'] = sum(thelist)
data = data.drop(columns = ['Forestry, fishing, and related activities', 'Mining, quarrying, and oil and gas extraction', 'Utilities', 
                            'Construction', 'Manufacturing', 'Wholesale trade', 'Transportation and warehousing', 
                            'Agriculture, forestry, fishing and hunting'])

data['Farm'] = data['Farm employment']
data = data.drop(columns = 'Farm employment')

In [25]:
data['Government %'] = percent(data['Government'], data['Total employment (number of jobs)'])
data['Retail & Hospitality %'] = percent(data['Retail & Hospitality'], data['Total employment (number of jobs)'])
data['Professional Services %'] = percent(data['Professional Services'], data['Total employment (number of jobs)'])
data['Industrial, Manufacturing, Utilities, & Logistics %'] = percent(data['Industrial, Manufacturing, Utilities, & Logistics'], 
                                                                      data['Total employment (number of jobs)'])
data['Farm %'] = percent(data['Farm'], data['Total employment (number of jobs)'])
data['White Collar %'] = percent(data['White Collar'], data['Total employment (number of jobs)'])
data['Blue Collar %'] = percent(data['Blue Collar'], data['Total employment (number of jobs)'])

In [26]:
data.reset_index()
data = data.drop(columns = ['Total employment (number of jobs)', 'Government', 'White Collar', 'Blue Collar', 'Retail & Hospitality', 
                            'Professional Services', 'Industrial, Manufacturing, Utilities, & Logistics', 'Farm'])
data = data.add_suffix(' BEA_2023')

In [27]:
data.head()

Industry,Government % BEA_2023,Retail & Hospitality % BEA_2023,Professional Services % BEA_2023,"Industrial, Manufacturing, Utilities, & Logistics % BEA_2023",Farm % BEA_2023,White Collar % BEA_2023,Blue Collar % BEA_2023
BEA Employment,7.240818,16.781027,40.147935,34.271531,1.558689,33.761578,33.936001


In [28]:
data.to_csv('IndustryEmployment_Current_AtaGlance_{}.csv'.format(county))

# Zillow

In [29]:
conn = sq.connect('../../../Data-Pipelines/Outputs/Dem_Transpo_Housing_Collection.db')
sql_query = pd.read_sql('SELECT * FROM [Zillow_HomeValue_Annual_Change]', conn)
data = pd.DataFrame(sql_query)
data = data.loc[(data['NAME'] == 'Wilson County, Tennessee')]
data = data.loc[(data['Year'] == '2024')].reset_index(drop = True)
data = data.drop(columns = ['NAME', 'Year', 'Time Frame', 'Home Value % Change', 'Home Value Change', 'GEO_ID', 'Source'])
data = data.add_suffix(' Year through June 2024')

In [30]:
data.head()

,Home Value Year through June 2024
0,481261.19058


In [31]:
data.to_csv('Zillow_AtaGlance_{}.csv'.format(county))

# Population Projections

In [35]:
conn = sq.connect('../../../Data-Pipelines/Outputs/WoodsandPooleandAffiliated.db')
sql_query = pd.read_sql('SELECT * FROM [smoothedtotalpop_2023PEP]', conn)
pop = pd.DataFrame(sql_query)
pop = pop.loc[pop['NAME'] == 'Wilson County, Tennessee']
# pop = pop[['NAME', ]]

In [48]:
#Historical Population
conn = sq.connect('../../../Data-Pipelines/Outputs/CensusBureau.db')
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_HistoricalPopulation_Annual_Change]', conn)
data = pd.DataFrame(sql_query)
data = data.loc[data['NAME'] == 'Wilson County, Tennessee']
data = data.loc[data['Time Frame'] == 'None']
data = data[['NAME', 'Year', 'Population']]
thelist = ['1950', '1960']
historicalpop = data.loc[data['Year'].isin(thelist)]

In [49]:
historicalpop.head()

,NAME,Year,Population
22923,"Wilson County, Tennessee",1950,26318.0
22924,"Wilson County, Tennessee",1960,27668.0


In [50]:
historicalpop = historicalpop.pivot(index = 'NAME', columns = 'Year', values = 'Population')

In [51]:
historicalpop.head()

Year,1950,1960
NAME,,
"Wilson County, Tennessee",26318.0,27668.0


In [52]:
pop.head()

,NAME,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023 Census,2023 WP,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
17,"Wilson County, Tennessee",36036.0,37381.0,38939.0,40558.0,42613.0,44579.0,46560.0,47556.0,50341.0,52409.0,55137.0,56227.0,56657.0,57051.0,57692.0,59330.0,60978.0,62197.0,64588.0,66122.0,67051.0,68019.0,69676.0,71087.0,73054.0,75373.0,77366.0,79760.0,82054.0,84618.0,87309.0,89220.0,91020.0,93033.0,94924.0,97139.0,99960.0,103697.0,106757.0,110474.0,112350.0,114681.0,116866.0,119275.0,122188.0,125459.0,128868.0,132958.0,137185.0,141321.0,144971.0,148645.0,152010.0,158555.0,163674.0,161278.0,166335.259259,169020.518519,171723.777778,174443.037037,177177.296296,179926.555556,182691.814815,185473.074074,188265.333333,191070.592593,193883.851852,196702.111111,199521.37037,202343.62963,205171.888889,208009.148148,210863.407407,213734.666667,216628.925926,219549.185185,222498.444444,225481.703704,228502.962963,231563.222222,234671.481481,237830.740741,241042.0,244399.0,247817.0,251295.0,254838.0,258451.0,262135.0,265892.0,269724.0,273631.0,277614.0


In [53]:
pop = pop[['NAME', '1970', '1980', '1990', '2000', '2010', '2020', '2025', '2030', '2035', '2040', '2045', '2050']]

In [54]:
pop = historicalpop.merge(pop, on = 'NAME')

In [55]:
pop.head()

,NAME,1950,1960,1970,1980,1990,2000,2010,2020,2025,2030,2035,2040,2045,2050
0,"Wilson County, Tennessee",26318.0,27668.0,37381.0,56227.0,68019.0,89220.0,114681.0,148645.0,169020.518519,182691.814815,196702.111111,210863.407407,225481.703704,241042.0


In [ ]:
data.to_csv('Population_Projections_AtaGlance_{}.csv'.format(county))